# Part Similarity Analysis Using Text Embeddings

## Business Objective

Identify similar parts across the parts catalog by analyzing textual descriptions. This enables:
- **Consolidation opportunities**: Find duplicate or near-duplicate parts for inventory reduction
- **Substitution recommendations**: Suggest alternatives when a part is unavailable
- **Procurement optimization**: Group similar parts for bulk purchasing

## Technical Approach

We use **text embeddings** (dense vector representations) generated by Snowflake Cortex's E5-base-v2 model to capture semantic meaning from part names, descriptions, and materials. Similarity between parts is computed via **cosine similarity** on their embedding vectors.

## Learning Objectives

After completing this notebook, you will understand:
1. How text embeddings encode semantic meaning into numerical vectors
2. Why cosine similarity is appropriate for comparing high-dimensional embeddings
3. How to use Snowflake Cortex functions for ML-powered text analysis
4. Best practices for fail-fast query execution in production notebooks

## Prerequisites

- **Mathematics**: Basic linear algebra (vectors, dot products, norms)
- **ML Concepts**: Understanding of embeddings and similarity metrics
- **Python**: Familiarity with pandas DataFrames and matplotlib
- **Domain**: General understanding of parts/inventory management

## Notebook Structure

| Section | Purpose |
|---------|----------|
| 1. Title & Objectives | Frame the problem and learning goals |
| 2. Environment Setup | Configure runtime, imports, and helpers |
| 3. Data Loading | Load part master data with validation |
| 4. Data Exploration | Analyze distributions and data quality |
| 5. Embedding Generation | Create vector representations of parts |
| 6. Similarity Computation | Calculate pairwise cosine similarity |
| 7. Evaluation | Analyze and visualize similarity results |
| 8. Production Output | Write results with verification |
| 9. Key Takeaways | Summary, limitations, and next steps |

## Output

- **`DATA_SCIENCE.PART_EMBEDDINGS`**: 768-dimensional embedding vectors for each part
- **`DATA_SCIENCE.PART_SIMILARITY_SCORES`**: Top-3 most similar parts for each source part

---

## 2. Environment Setup

This section configures the runtime environment including:
- Package imports for data manipulation and visualization
- Snowflake session initialization
- Dark theme visualization settings (Snowflake-inspired, colorblind-safe)
- Fail-fast query execution helper for production reliability

In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================
# Standard library
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Snowflake
from snowflake.snowpark.context import get_active_session

In [ ]:
# =============================================================================
# VISUALIZATION THEME: Snowflake-inspired dark mode
# =============================================================================
# Soft dark background (#121212) reduces eye strain vs pure black.
# Off-white text (#E5E5E7) reduces glare vs pure white.
# Colorblind-safe palette avoids red/green distinctions.

plt.style.use('dark_background')
plt.rcParams.update({
    # Background colors (soft dark gray)
    'figure.facecolor': '#121212',
    'axes.facecolor': '#121212',
    
    # Text colors (off-white)
    'text.color': '#E5E5E7',
    'axes.labelcolor': '#E5E5E7',
    'xtick.color': '#A1A1A6',
    'ytick.color': '#A1A1A6',
    
    # Grid and axes (subtle)
    'axes.edgecolor': '#3A3A3C',
    'grid.color': '#2C2C2E',
    'grid.alpha': 0.6,
    
    # Figure quality
    'figure.dpi': 150,
    'savefig.dpi': 200,
    'figure.figsize': (10, 6),
    
    # Typography
    'font.family': 'sans-serif',
    'font.size': 11,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
})

# Colorblind-safe palette for dark backgrounds
SNOWFLAKE_COLORS = ['#64D2FF', '#FF9F0A', '#5AC8FA', '#FFD60A', '#11567F']
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=SNOWFLAKE_COLORS)

print("Visualization theme configured (dark mode, colorblind-safe)")

In [ ]:
# =============================================================================
# SNOWFLAKE SESSION
# =============================================================================
# Get session from Snowflake Notebooks context (do not create new connection)

session = get_active_session()
print(f"Session established")
print(f"  Current database: {session.get_current_database()}")
print(f"  Current schema: {session.get_current_schema()}")
print(f"  Current warehouse: {session.get_current_warehouse()}")

In [ ]:
# =============================================================================
# FAIL-FAST QUERY HELPER
# =============================================================================
# All SQL queries must raise exceptions on failure - never suppress errors.
# This ensures the notebook fails immediately if data issues occur.

def execute_query(query: str, name: str = "query") -> pd.DataFrame:
    """
    Execute a SQL query with fail-fast error handling.
    
    Args:
        query: SQL query string to execute
        name: Descriptive name for error messages
    
    Returns:
        DataFrame with query results
    
    Raises:
        RuntimeError: If query fails or returns None
    """
    try:
        result = session.sql(query).to_pandas()
        if result is None:
            raise RuntimeError(f"Query '{name}' returned None")
        return result
    except Exception as e:
        raise RuntimeError(f"Query '{name}' failed: {e}") from e


def execute_statement(query: str, name: str = "statement") -> None:
    """
    Execute a SQL statement (DDL/DML) with fail-fast error handling.
    
    Args:
        query: SQL statement to execute
        name: Descriptive name for error messages
    
    Raises:
        RuntimeError: If statement fails
    """
    try:
        session.sql(query).collect()
    except Exception as e:
        raise RuntimeError(f"Statement '{name}' failed: {e}") from e


print("Query helpers defined: execute_query(), execute_statement()")

---

## 3. Data Loading

Load part master data from `ATOMIC.PART_MASTER` and validate that required columns exist and the dataset is not empty.

In [ ]:
# =============================================================================
# LOAD PART MASTER DATA
# =============================================================================
# We load a sample of parts for embedding generation.
# The LIMIT clause controls processing scope for development/testing.

SAMPLE_LIMIT = 5000  # Number of parts to process

parts_df = execute_query(
    f"""
    SELECT 
        GLOBAL_ID,
        PART_NAME,
        PART_DESCRIPTION,
        MATERIAL
    FROM ATOMIC.PART_MASTER
    LIMIT {SAMPLE_LIMIT}
    """,
    name="load_part_master"
)

# Validate data was loaded
if len(parts_df) == 0:
    raise RuntimeError("ATOMIC.PART_MASTER is empty - cannot proceed")

# Validate required columns exist
required_columns = ['GLOBAL_ID', 'PART_NAME', 'PART_DESCRIPTION', 'MATERIAL']
missing = set(required_columns) - set(parts_df.columns)
if missing:
    raise RuntimeError(f"Missing required columns: {missing}")

print(f"Loaded {len(parts_df):,} parts from ATOMIC.PART_MASTER")
print(f"\nColumns: {list(parts_df.columns)}")

---

## 4. Data Exploration

Before generating embeddings, we analyze the input data to understand:
- Distribution of text lengths (affects embedding quality)
- Missing value rates (null descriptions will produce poor embeddings)
- Material distribution (categorical feature included in embedding)

In [ ]:
# =============================================================================
# DATA QUALITY ANALYSIS
# =============================================================================
# Check for missing values that could degrade embedding quality

# Calculate missing value rates
missing_counts = parts_df.isnull().sum()
missing_pct = (missing_counts / len(parts_df) * 100).round(2)

print("Missing Value Analysis:")
print("=" * 40)
for col in required_columns:
    count = missing_counts[col]
    pct = missing_pct[col]
    status = "" if pct == 0 else " (may affect embedding quality)"
    print(f"  {col}: {count:,} missing ({pct}%){status}")

# Calculate text lengths for non-null values
parts_df['text_length'] = (
    parts_df['PART_NAME'].fillna('').str.len() +
    parts_df['PART_DESCRIPTION'].fillna('').str.len() +
    parts_df['MATERIAL'].fillna('').str.len()
)

print(f"\nText Length Statistics (combined name + description + material):")
print(f"  Min: {parts_df['text_length'].min()} characters")
print(f"  Max: {parts_df['text_length'].max()} characters")
print(f"  Mean: {parts_df['text_length'].mean():.1f} characters")
print(f"  Median: {parts_df['text_length'].median():.1f} characters")

In [ ]:
# =============================================================================
# DATA EXPLORATION: Visualizations
# =============================================================================
# Understanding text length distribution helps interpret embedding quality.
# Very short texts may lack semantic content; very long texts may be truncated.

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Text length distribution
ax1 = axes[0]
ax1.hist(parts_df['text_length'], bins=50, color='#64D2FF', edgecolor='#121212', alpha=0.8)
ax1.axvline(x=parts_df['text_length'].median(), color='#FF9F0A', linestyle='--', 
            linewidth=2, label=f"Median: {parts_df['text_length'].median():.0f}")
ax1.set_xlabel('Combined Text Length (characters)')
ax1.set_ylabel('Number of Parts')
ax1.set_title('Distribution of Part Text Lengths')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Top materials by count
ax2 = axes[1]
material_counts = parts_df['MATERIAL'].fillna('(Unknown)').value_counts().head(10)
bars = ax2.barh(range(len(material_counts)), material_counts.values, color='#5AC8FA', alpha=0.8)
ax2.set_yticks(range(len(material_counts)))
ax2.set_yticklabels(material_counts.index)
ax2.set_xlabel('Number of Parts')
ax2.set_title('Top 10 Materials by Part Count')
ax2.invert_yaxis()  # Highest at top
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('/tmp/part_data_exploration.png', dpi=150, bbox_inches='tight', facecolor='#121212')
plt.show()

print(f"\nUnique materials: {parts_df['MATERIAL'].nunique():,}")

---

## 5. Embedding Generation

### What are Text Embeddings?

**Text embeddings** are dense vector representations that encode semantic meaning of text into numerical form. Unlike sparse representations (e.g., one-hot encoding or TF-IDF), embeddings place semantically similar texts close together in vector space.

### Why E5-base-v2?

We use Snowflake Cortex's `e5-base-v2` model because:
1. **Optimized for similarity search**: E5 models are trained with contrastive learning specifically for retrieval/similarity tasks
2. **768-dimensional output**: Provides rich semantic representation without excessive dimensionality
3. **Native Snowflake integration**: No data movement required; embeddings computed in-warehouse

### Mathematical Formulation

The embedding function maps text to a 768-dimensional unit vector:

$$\mathbf{e} = \text{E5}(\text{text}) \in \mathbb{R}^{768}, \quad ||\mathbf{e}|| = 1$$

We concatenate part attributes before embedding to capture the full semantic context:

$$\text{input} = \text{PART\_NAME} \oplus \text{PART\_DESCRIPTION} \oplus \text{MATERIAL}$$

### Architecture

```
PART_MASTER Table
       |
       v
[CONCAT: name + description + material]
       |
       v
[EMBED_TEXT_768: E5-base-v2 model]
       |
       v
PART_EMBEDDINGS Table (GLOBAL_ID, EMBEDDING[768])
```

In [ ]:
# =============================================================================
# GENERATE PART EMBEDDINGS
# =============================================================================
# Create 768-dimensional embedding vectors for each part using Snowflake Cortex.
# The embedding captures semantic meaning from the concatenated text fields.

embedding_sql = f"""
CREATE OR REPLACE TABLE DATA_SCIENCE.PART_EMBEDDINGS AS
SELECT
    GLOBAL_ID,
    SNOWFLAKE.CORTEX.EMBED_TEXT_768(
        'e5-base-v2',
        CONCAT(
            COALESCE(PART_NAME, ''), ' ',
            COALESCE(PART_DESCRIPTION, ''), ' ',
            COALESCE(MATERIAL, '')
        )
    ) AS EMBEDDING  -- (768,) vector per part
FROM ATOMIC.PART_MASTER
LIMIT {SAMPLE_LIMIT}
"""

print("Generating embeddings (this may take a few minutes)...")
execute_statement(embedding_sql, name="create_part_embeddings")

# Verify embeddings were created
embedding_count_df = execute_query(
    "SELECT COUNT(*) AS CNT FROM DATA_SCIENCE.PART_EMBEDDINGS",
    name="verify_embeddings"
)
embedding_count = int(embedding_count_df['CNT'].iloc[0])

if embedding_count == 0:
    raise RuntimeError("No embeddings were generated - check input data")

print(f"Generated {embedding_count:,} part embeddings")
print(f"  Table: DATA_SCIENCE.PART_EMBEDDINGS")
print(f"  Dimensions: 768 per embedding")

---

## 6. Similarity Computation

### What is Cosine Similarity?

**Cosine similarity** measures the angle between two vectors, ignoring their magnitudes. For unit vectors (like E5 embeddings), it equals the dot product:

$$\text{similarity}(\mathbf{a}, \mathbf{b}) = \cos(\theta) = \frac{\mathbf{a} \cdot \mathbf{b}}{||\mathbf{a}|| \cdot ||\mathbf{b}||} = \mathbf{a} \cdot \mathbf{b}$$

### Why Cosine Similarity for Embeddings?

1. **Scale invariant**: Text length doesn't bias similarity (unlike Euclidean distance)
2. **Bounded output**: Values in [-1, 1], where 1 = identical, 0 = orthogonal, -1 = opposite
3. **Computationally efficient**: Dot product is fast for dense vectors

### Output Schema

We store the **top-3 most similar parts** for each source part:

| Column | Type | Description |
|--------|------|-------------|
| SOURCE_GLOBAL_ID | VARCHAR | The reference part |
| TARGET_GLOBAL_ID | VARCHAR | A similar part |
| SIMILARITY_SCORE | FLOAT | Cosine similarity (0-100 scale) |
| MATCH_REASON | VARCHAR | Method used for matching |

### Why Top-3?

Storing all pairwise similarities would create N² rows (25M rows for 5K parts). Top-3 provides actionable recommendations while keeping the output manageable.

In [ ]:
# =============================================================================
# COMPUTE PAIRWISE SIMILARITY SCORES
# =============================================================================
# For each part, find the top-3 most similar parts based on embedding cosine
# similarity. We exclude self-matches (a.GLOBAL_ID <> b.GLOBAL_ID).

TOP_K = 3  # Number of similar parts to keep per source

# Truncate existing scores for idempotent execution
execute_statement(
    "TRUNCATE TABLE IF EXISTS DATA_SCIENCE.PART_SIMILARITY_SCORES",
    name="truncate_similarity_scores"
)

# Compute and insert similarity scores
similarity_sql = f"""
INSERT INTO DATA_SCIENCE.PART_SIMILARITY_SCORES 
    (SOURCE_GLOBAL_ID, TARGET_GLOBAL_ID, SIMILARITY_SCORE, MATCH_REASON)
WITH pairs AS (
    SELECT
        a.GLOBAL_ID AS SOURCE_GLOBAL_ID,
        b.GLOBAL_ID AS TARGET_GLOBAL_ID,
        -- Cosine similarity scaled to 0-100 for readability
        VECTOR_COSINE_SIMILARITY(a.EMBEDDING, b.EMBEDDING) * 100 AS SIMILARITY_SCORE,
        'E5 embedding cosine similarity' AS MATCH_REASON
    FROM DATA_SCIENCE.PART_EMBEDDINGS a
    JOIN DATA_SCIENCE.PART_EMBEDDINGS b
        ON a.GLOBAL_ID <> b.GLOBAL_ID  -- Exclude self-matches
)
SELECT *
FROM pairs
-- Keep only top-K matches per source part
QUALIFY ROW_NUMBER() OVER (
    PARTITION BY SOURCE_GLOBAL_ID 
    ORDER BY SIMILARITY_SCORE DESC
) <= {TOP_K}
"""

print(f"Computing top-{TOP_K} similarity scores per part...")
execute_statement(similarity_sql, name="insert_similarity_scores")
print("Similarity computation complete")

---

## 7. Evaluation

Analyze the distribution of similarity scores to understand:
- How similar are the "most similar" parts? (Are embeddings discriminative?)
- What score thresholds indicate strong vs weak matches?
- Are there outliers or unexpected patterns?

In [ ]:
# =============================================================================
# LOAD SIMILARITY RESULTS FOR ANALYSIS
# =============================================================================

scores_df = execute_query(
    """
    SELECT 
        SOURCE_GLOBAL_ID,
        TARGET_GLOBAL_ID,
        SIMILARITY_SCORE,
        MATCH_REASON
    FROM DATA_SCIENCE.PART_SIMILARITY_SCORES
    """,
    name="load_similarity_scores"
)

if len(scores_df) == 0:
    raise RuntimeError("No similarity scores found - check computation step")

print(f"Loaded {len(scores_df):,} similarity score records")
print(f"  Unique source parts: {scores_df['SOURCE_GLOBAL_ID'].nunique():,}")
print(f"  Score range: {scores_df['SIMILARITY_SCORE'].min():.2f} to {scores_df['SIMILARITY_SCORE'].max():.2f}")

In [ ]:
# =============================================================================
# SIMILARITY SCORE DISTRIBUTION ANALYSIS
# =============================================================================
# Visualize the distribution of similarity scores to establish thresholds
# and understand embedding discriminativeness.

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Define interpretation thresholds
HIGH_THRESHOLD = 80  # Strong match
MEDIUM_THRESHOLD = 60  # Moderate match

# Plot 1: Overall distribution with thresholds
ax1 = axes[0]
ax1.hist(scores_df['SIMILARITY_SCORE'], bins=50, color='#64D2FF', edgecolor='#121212', alpha=0.8)
ax1.axvline(x=HIGH_THRESHOLD, color='#FF9F0A', linestyle='--', linewidth=2, label=f'High ({HIGH_THRESHOLD}+)')
ax1.axvline(x=MEDIUM_THRESHOLD, color='#FFD60A', linestyle='--', linewidth=2, label=f'Medium ({MEDIUM_THRESHOLD}+)')
ax1.set_xlabel('Similarity Score (0-100)')
ax1.set_ylabel('Number of Pairs')
ax1.set_title('Distribution of All Similarity Scores')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Distribution by rank (1st, 2nd, 3rd most similar)
ax2 = axes[1]
scores_df['RANK'] = scores_df.groupby('SOURCE_GLOBAL_ID')['SIMILARITY_SCORE'].rank(ascending=False, method='first')
colors = ['#64D2FF', '#FF9F0A', '#5AC8FA']
for rank in [1, 2, 3]:
    rank_scores = scores_df[scores_df['RANK'] == rank]['SIMILARITY_SCORE']
    ax2.hist(rank_scores, bins=30, alpha=0.6, label=f'Rank {int(rank)}', color=colors[rank-1])
ax2.set_xlabel('Similarity Score (0-100)')
ax2.set_ylabel('Number of Pairs')
ax2.set_title('Score Distribution by Similarity Rank')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Box plot by rank
ax3 = axes[2]
rank_data = [scores_df[scores_df['RANK'] == r]['SIMILARITY_SCORE'].values for r in [1, 2, 3]]
bp = ax3.boxplot(rank_data, labels=['1st', '2nd', '3rd'], patch_artist=True)
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax3.set_xlabel('Similarity Rank')
ax3.set_ylabel('Similarity Score (0-100)')
ax3.set_title('Score Range by Rank')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/tmp/similarity_distribution.png', dpi=150, bbox_inches='tight', facecolor='#121212')
plt.show()

# Summary statistics
high_count = (scores_df['SIMILARITY_SCORE'] >= HIGH_THRESHOLD).sum()
medium_count = ((scores_df['SIMILARITY_SCORE'] >= MEDIUM_THRESHOLD) & 
                (scores_df['SIMILARITY_SCORE'] < HIGH_THRESHOLD)).sum()
low_count = (scores_df['SIMILARITY_SCORE'] < MEDIUM_THRESHOLD).sum()

print(f"\nSimilarity Score Summary:")
print(f"  High (>={HIGH_THRESHOLD}): {high_count:,} pairs ({high_count/len(scores_df)*100:.1f}%)")
print(f"  Medium ({MEDIUM_THRESHOLD}-{HIGH_THRESHOLD}): {medium_count:,} pairs ({medium_count/len(scores_df)*100:.1f}%)")
print(f"  Low (<{MEDIUM_THRESHOLD}): {low_count:,} pairs ({low_count/len(scores_df)*100:.1f}%)")

In [ ]:
# =============================================================================
# SAMPLE HIGH-SIMILARITY PAIRS
# =============================================================================
# Examine some of the highest-scoring pairs to validate embedding quality.

top_pairs_df = execute_query(
    f"""
    SELECT 
        s.SOURCE_GLOBAL_ID,
        p1.PART_NAME AS SOURCE_PART_NAME,
        p1.MATERIAL AS SOURCE_MATERIAL,
        s.TARGET_GLOBAL_ID,
        p2.PART_NAME AS TARGET_PART_NAME,
        p2.MATERIAL AS TARGET_MATERIAL,
        s.SIMILARITY_SCORE
    FROM DATA_SCIENCE.PART_SIMILARITY_SCORES s
    JOIN ATOMIC.PART_MASTER p1 ON s.SOURCE_GLOBAL_ID = p1.GLOBAL_ID
    JOIN ATOMIC.PART_MASTER p2 ON s.TARGET_GLOBAL_ID = p2.GLOBAL_ID
    WHERE s.SIMILARITY_SCORE >= {HIGH_THRESHOLD}
    ORDER BY s.SIMILARITY_SCORE DESC
    LIMIT 10
    """,
    name="top_similarity_pairs"
)

print(f"Top 10 Highest-Similarity Pairs (score >= {HIGH_THRESHOLD}):")
print("=" * 80)
if len(top_pairs_df) > 0:
    for _, row in top_pairs_df.iterrows():
        print(f"\nScore: {row['SIMILARITY_SCORE']:.2f}")
        print(f"  Source: {row['SOURCE_PART_NAME']} ({row['SOURCE_MATERIAL']})")
        print(f"  Target: {row['TARGET_PART_NAME']} ({row['TARGET_MATERIAL']})")
else:
    print(f"No pairs found with similarity >= {HIGH_THRESHOLD}")

---

## 8. Production Output Verification

Verify that the output tables were written correctly and contain expected data.

In [ ]:
# =============================================================================
# OUTPUT VERIFICATION
# =============================================================================
# Validate that output tables contain expected row counts.

# Verify embeddings table
embeddings_verify = execute_query(
    "SELECT COUNT(*) AS CNT FROM DATA_SCIENCE.PART_EMBEDDINGS",
    name="verify_embeddings_final"
)
embeddings_count = int(embeddings_verify['CNT'].iloc[0])

# Verify similarity scores table
scores_verify = execute_query(
    "SELECT COUNT(*) AS CNT FROM DATA_SCIENCE.PART_SIMILARITY_SCORES",
    name="verify_scores_final"
)
scores_count = int(scores_verify['CNT'].iloc[0])

# Expected: TOP_K scores per embedding
expected_scores = embeddings_count * TOP_K

print("Output Table Verification:")
print("=" * 50)
print(f"\n DATA_SCIENCE.PART_EMBEDDINGS")
print(f"    Rows: {embeddings_count:,}")
print(f"    Status: {'PASS' if embeddings_count > 0 else 'FAIL'}")

print(f"\n DATA_SCIENCE.PART_SIMILARITY_SCORES")
print(f"    Rows: {scores_count:,}")
print(f"    Expected: ~{expected_scores:,} (top-{TOP_K} per part)")
print(f"    Status: {'PASS' if scores_count > 0 else 'FAIL'}")

if embeddings_count == 0 or scores_count == 0:
    raise RuntimeError("Output verification failed - tables are empty")

print("\nAll output tables verified successfully")

---

## 9. Key Takeaways & Interpretation Guide

### What the Model Learned

The E5 embedding model captures **semantic similarity** between parts based on their textual descriptions. Parts with:
- Similar names, descriptions, or materials cluster together in embedding space
- Different terminology but same meaning (e.g., "bolt" vs "fastener") may still match
- Completely different purposes have low similarity even if they share words

### Interpretation Guidelines

| Score Range | Interpretation | Recommended Action |
|-------------|----------------|--------------------|
| 80-100 | **Strong match**: Very similar parts | Investigate for consolidation |
| 60-80 | **Moderate match**: Related parts | Consider as substitutes |
| 40-60 | **Weak match**: Some overlap | Review before using |
| 0-40 | **Poor match**: Likely different | Not recommended as similar |

### Limitations & Considerations

1. **Text quality dependency**: Poor or missing descriptions produce poor embeddings. Garbage in = garbage out.

2. **No dimensional awareness**: The model doesn't understand that "10mm bolt" and "12mm bolt" are different sizes - they will have high similarity because the text is similar.

3. **Sample limitation**: This notebook processes only 5,000 parts. Production deployment should remove or increase the LIMIT clause.

4. **Top-K truncation**: Only storing top-3 matches means some useful moderate-similarity pairs are discarded.

### Mathematical Recap

**Embedding**: $\mathbf{e} = \text{E5}(\text{text}) \in \mathbb{R}^{768}$

**Cosine Similarity**: $\text{sim}(\mathbf{a}, \mathbf{b}) = \frac{\mathbf{a} \cdot \mathbf{b}}{||\mathbf{a}|| \cdot ||\mathbf{b}||}$

**Score Scaling**: $\text{score} = \text{sim} \times 100$ (for readability)

### Further Learning Resources

- [Snowflake Cortex LLM Functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions)
- [E5 Embedding Model Paper](https://arxiv.org/abs/2212.03533)
- [Understanding Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity)

### Next Steps

1. **Increase sample size**: Remove LIMIT for production to process all parts
2. **Add metadata filtering**: Pre-filter by category before computing similarity
3. **Human validation**: Review high-scoring pairs with domain experts
4. **Integrate with procurement**: Connect similarity scores to purchasing workflows